In [ ]:
#Create local file with the following data

Complete the following code using pyspark RDDs

movie.csv
movieId,moviename,director,year
200,shawshank,FrankD,1994
201,saw,JamesW,2004
202,avengers,JossW,2012
204,mi,ChristopherMcq,2018
205,wrongturn,DeclanO,2011

award.csv
catalogId,movieId,awards
11,200,2
12,200,3
13,201,1
14,202,3
15,202,5
16,204,4
17,205,1

In [1]:
#Import libraries & Create SparkSession
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488490 sha256=2a2e1b2bf34f245f6c4d251b05eb0747da25e2f6070f4ecac7785e6b50604ca9
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [10]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
lti_spark=SparkSession.builder.appName("sound").getOrCreate()

In [6]:
#Create two dataframes using respective csv files from local storage
%%writefile movie.csv
movieId,moviename,director,year
200,shawshank,FrankD,1994
201,saw,JamesW,2004
202,avengers,JossW,2012
204,mi,ChristopherMcq,2018
205,wrongturn,DeclanO,2011

Writing movie.csv


In [7]:
%%writefile awards.csv
catalogId,movieId,awards
11,200,2
12,200,3
13,201,1
14,202,3
15,202,5
16,204,4
17,205,1

Writing awards.csv


In [8]:
m_df=lti_spark.read.csv("/content/movie.csv",header=True)
a_df=lti_spark.read.csv("/content/awards.csv",header=True)

In [11]:
#Create a new column called "movieInfo" in movies dataframe which includes movieName & Director
m_df.withColumn("movieInfo",concat_ws(" ","moviename","director")).show()

+-------+---------+--------------+----+-----------------+
|movieId|moviename|      director|year|        movieInfo|
+-------+---------+--------------+----+-----------------+
|    200|shawshank|        FrankD|1994| shawshank FrankD|
|    201|      saw|        JamesW|2004|       saw JamesW|
|    202| avengers|         JossW|2012|   avengers JossW|
|    204|       mi|ChristopherMcq|2018|mi ChristopherMcq|
|    205|wrongturn|       DeclanO|2011|wrongturn DeclanO|
+-------+---------+--------------+----+-----------------+



In [19]:
#Calculate total number of awards for each movie & display - movieID, movienName, awardsCount
a=m_df.join(a_df,"movieId").groupby("moviename","movieId").agg(sum("awards").alias("awardcount"))
a.show()

+---------+-------+----------+
|moviename|movieId|awardcount|
+---------+-------+----------+
|wrongturn|    205|       1.0|
|shawshank|    200|       5.0|
|       mi|    204|       4.0|
|      saw|    201|       1.0|
| avengers|    202|       8.0|
+---------+-------+----------+



In [22]:
#Get top three movies on the basis of number of awards
a.sort("awardcount",ascending=False).show(3)

+---------+-------+----------+
|moviename|movieId|awardcount|
+---------+-------+----------+
| avengers|    202|       8.0|
|shawshank|    200|       5.0|
|       mi|    204|       4.0|
+---------+-------+----------+
only showing top 3 rows



In [27]:
#Get top three latest movies
m_df.sort(col("year"),ascending=False).show(3)

+-------+---------+--------------+----+
|movieId|moviename|      director|year|
+-------+---------+--------------+----+
|    204|       mi|ChristopherMcq|2018|
|    202| avengers|         JossW|2012|
|    205|wrongturn|       DeclanO|2011|
+-------+---------+--------------+----+
only showing top 3 rows

